In [1]:
import cv2 as cv
import numpy as np
import pytesseract
from transformers import pipeline
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
#summarizer = pipeline('summarization')
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [2]:
def findPoints(contour):
    epsilon = 0.02*cv.arcLength(a, True)
    approx = cv.approxPolyDP(a, epsilon, True)
    if len(approx) != 4:
        return False, None
    if len(approx) == 4:
        draw.append(a)
        points = approx.reshape((4, 2))
        pointsNew = np.zeros((4, 1, 2), np.int32)
        add = points.sum(1)
        diff = np.diff(points, axis = 1)
        pointsNew[0] = points[np.argmin(add)]
        pointsNew[1] = points[np.argmin(diff)]
        pointsNew[2] = points[np.argmax(diff)]         
        pointsNew[3] = points[np.argmax(add)]
        return True, pointsNew

In [3]:
def getContours(frame, top):
    imgray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    _, thresh = cv.threshold(imgray, 100, 255, 0)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    acceptable = sorted(contours, key = cv.contourArea, reverse=True)
    return tuple(acceptable[:top])

In [4]:
def screenDisplay(morphs, frame):
    points_A = np.float32(morphs[0])
    points_B = np.float32([[0,0], [700,0], [0,1200], [700,1200]])
    M = cv.getPerspectiveTransform(points_A, points_B)
    warped = cv.warpPerspective(frame, M, (700,1200))
    warpedgray = cv.cvtColor(warped, cv.COLOR_BGR2GRAY)
    _, thresher = cv.threshold(warpedgray, 180, 255, 0)
    kernel_sharpening = np.array([[-1,-1,-1], 
                                 [-1,9,-1], 
                                 [-1,-1,-1]])
    return cv.filter2D(thresher, -1, kernel_sharpening)

In [5]:
cap = cv.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    acceptable = getContours(frame, 2)
    draw = []
    morphs = []
    for a in acceptable:
            available, points = findPoints(a)
            if available:
                morphs.append(points)
                for loc in points:
                    cv.circle(frame, loc[0], 15, (0, 0, 255), -1)
        
    if ret:
        cv.drawContours(frame, tuple(draw), -1, (0,255,0), 3)
        cv.imshow("Shapes", frame)
        if draw:
            sharpened = screenDisplay(morphs, frame)
            cv.imshow('sharpened', sharpened)
    if cv.waitKey(1) & 0xFF  == ord('q'):
        break
    elif cv.waitKey(1) & 0xFF  == ord(' '):
        text = pytesseract.image_to_string(sharpened, lang='eng', config='--psm 6 --oem 3')
        text = text.rstrip('\n')
        print(text)
        
        input_ids = tokenizer.encode(text, return_tensors='pt') #pytorch
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)

        #summary = summarizer(text, max_length=130, min_length=15, do_sample=False)
        print(3*"--" + " SUMMARY " + 3*"--")
        print('\n' + summary)
        
        break
        
cap.release()
cv.destroyAllWindows()


Martha and George were raising.
(Martha's son Jacky had four children
before he died, and the two youngest.
ten-year-old Eleanor Parke Custis”
called -Nelly”—and eight-year-old
George Washington Parke Custis—
called -wWashy”—bad moved in with the
Washingtons) Ona already knew that
during her “free” time. she would be
bel ping Moll with the grandchildren.
Likely, she never thought her life in
New York would be anything but ex-
hansting. It was 3 privilege of freedom
to be able to think about things like
ralking to 2 friend, enjoying leisure
time, oF appreciating the curiosities of
a new City-
The other slaves chosen to go to New
York were all male Besides William
Lee (who was already in New York with
the president. qwo men, aamed Giles
aod Pease erere | ing to serve as postil-
ooh iets een bingrons. That is, they
— , 2 SOT carriages
, ber slaves
York were
sie
r ae: be waiters
v i doed by
~ y that Christo-
. ” fy,member
gary choice.
ee
m7 a
------ SUMMARY ------

The slaves chosen to go 